<a href="https://colab.research.google.com/github/sycho2003/20252R0136COSE36203/blob/main/%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5_Term_Project_%EC%99%9C%EA%B3%A1%EB%B3%84_%EC%9D%B4%EC%A7%84_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. All-or-nothing Thinking

## 1. 모델 정의

### 1-1. 기본 준비

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data1 = pd.read_csv('d01_preprocessed_revised.csv')
data2 = pd.read_csv('d02_preprocessed.csv')
data3 = pd.read_csv('d03_preprocessed.csv')

### 1-2. 데이터 증강 및 구조화

In [ ]:
for idx, i in enumerate(data2['thought']):
    if type(i) != str:
        data2['thought'][idx] = ''

In [ ]:
# has_distortion == 1 필터링

data1 = data1[data1['has_distortion'] == 1].reset_index(drop=True)
data2 = data2[data2['has_distortion'] == 1].reset_index(drop=True)
data3 = data3[data3['has_distortion'] == 1].reset_index(drop=True)

In [ ]:
data1_1 = data1['situation']+' '+data1['thought']
data2_1 = data2['situation']
data3_1 = data3['situation']+' '+data3['thought']

In [ ]:
data1_1.drop_duplicates(inplace = True)
data2_1.drop_duplicates(inplace = True)
data3_1.drop_duplicates(inplace = True)

In [ ]:
def normalize_text(s):
    # Removing articles and punctuation, and standardizing whitespace
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


In [ ]:
from transformers import BertTokenizer, BertModel, BertConfig

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.to(device)
bert_config = BertConfig.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Embedding

def tokenize_and_pad(data, tokenizer, max_len=512):
    tokenized_data = []
    for text in data:
        encoded = tokenizer(normalize_text(text), return_tensors="pt", padding='max_length', truncation=True, max_length=max_len)
        tokenized_data.append(encoded)
    return tokenized_data

data1_1_encoded = tokenize_and_pad(data1_1, tokenizer)
data2_1_encoded = tokenize_and_pad(data2_1, tokenizer)
data3_1_encoded = tokenize_and_pad(data3_1, tokenizer)

In [ ]:
data1.columns

Index(['situation', 'thought', 'reframe', 'has_distortion',
       'all-or-nothing thinking', 'comparing and despairing',
       'disqualifying the positive', 'emotional reasoning', 'fortune telling',
       'labeling', 'magnification', 'mind reading', 'overgeneralizing',
       'should statements', 'mental filter', 'personalization and blaming'],
      dtype='object')

In [ ]:
# Add labels
data1_1_labels = list(data1['all-or-nothing thinking'][data1_1.index])
data2_1_labels = list(data2['all-or-nothing thinking'][data2_1.index])
data3_1_labels = list(data3['all-or-nothing thinking'][data3_1.index])

In [ ]:
# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

In [ ]:
class CustomDatasetWithLabels(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {"input_ids": self.data[idx]['input_ids'].squeeze(),
                "attention_mask": self.data[idx]['attention_mask'].squeeze(),
                "y": self.labels[idx]}

In [ ]:
dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

### 1-3. 모델 평가 함수 정의

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate(model, dataloader, device="cpu"):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            y = batch["y"].to(device)

            # Get embeddings from the BERT model
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

            logits = model(embeddings)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(y.cpu().tolist())

    acc = accuracy_score(all_labels, all_preds)
    f1_macro = f1_score(all_labels, all_preds, average="macro", zero_division=0)

    return {"accuracy": acc, "f1_macro": f1_macro}

In [ ]:
# Classifier that uses label embeddings to make predictions
class InnerProductClassifier(nn.Module):
    def __init__(self, input_dim, label_embeddings, trainable_label_emb=True):
        super().__init__()
        # Project input features into the same dimension as label embeddings
        self.proj = nn.Linear(input_dim, label_embeddings.size(1))

        if trainable_label_emb:
            # Label embeddings are trainable parameters
            self.label_emb = nn.Parameter(label_embeddings.clone())
        else:
            # Label embeddings are fixed (not updated during training)
            self.register_buffer("label_emb", label_embeddings.clone())

    def forward(self, x):
        # Project input feature vectors
        x_proj = self.proj(x)
        # Compute logits as similarity with each label embedding
        logits = torch.matmul(x_proj, self.label_emb.T)
        return logits

### 1-4. 모델 생성

In [ ]:
# Instantiate the InnerProductClassifier model
model = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

## 2. 모델 학습

In [ ]:
EPOCHS = 10

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   4%|▍         | 1/26 [00:02<01:00,  2.41s/it]

Loss: 1.4566470384597778


Epoch 1:   8%|▊         | 2/26 [00:04<00:46,  1.94s/it]

Loss: 18.050392150878906


Epoch 1:  12%|█▏        | 3/26 [00:05<00:41,  1.81s/it]

...

Epoch 10:  96%|█████████▌| 25/26 [00:47<00:01,  1.91s/it]

Loss: 0.5097233653068542


Epoch 10: 100%|██████████| 26/26 [00:48<00:00,  1.87s/it]

Loss: 0.29963621497154236


Epoch 10 Validation Accuracy: 0.9064039408866995, F1-macro: 0.5229437229437229


## 3. 모델 테스트

In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8480392156862745, F1-macro: 0.5229437229437229


In [ ]:
distortion_types = [
    'all-or-nothing thinking', 'comparing and despairing',
    'disqualifying the positive', 'emotional reasoning', 'fortune telling',
    'labeling', 'magnification', 'mind reading', 'overgeneralizing',
    'should statements', 'mental filter', 'personalization and blaming'
]

In [ ]:
results_df = pd.DataFrame({
    "distortion_type": distortion_types,
    "test_accuracy": [np.nan] * 12,
    "f1_macro": [np.nan] * 12
})

In [ ]:
current_type = 'all-or-nothing thinking'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 2. Comparing and Despairing

In [ ]:
# Add labels
data1_1_labels = list(data1['comparing and despairing'][data1_1.index])
# comparing and despairing은 data1에만 있음.

# Merging Data
data_encoded = data1_1_encoded
data_labels = data1_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 10

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:  33%|███▎      | 1/3 [00:01<00:03,  1.89s/it]

Loss: 10.964028358459473


Epoch 1:  67%|██████▋   | 2/3 [00:03<00:01,  1.90s/it]

Loss: 1.0615003108978271


Epoch 1: 100%|██████████| 3/3 [00:05<00:00,  1.85s/it]

...

Epoch 10:  67%|██████▋   | 2/3 [00:03<00:01,  1.90s/it]

Loss: 0.39907124638557434


Epoch 10: 100%|██████████| 3/3 [00:05<00:00,  1.85s/it]

Loss: 0.29249656200408936


Epoch 10 Validation Accuracy: 0.8260869565217391, F1-macro: 0.4523809523809524


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.84, F1-macro: 0.4523809523809524


In [ ]:
current_type = 'comparing and despairing'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 3. Disqualifying the Positive

In [ ]:
# Add labels
data1_1_labels = list(data1['disqualifying the positive'][data1_1.index])
data2_1_labels = list(data2['disqualifiying the positive'][data2_1.index]) # 자료에서 오타 있었음!
data3_1_labels = list(data3['disqualifying the positive'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 10

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   4%|▍         | 1/26 [00:01<00:47,  1.89s/it]

Loss: 0.07945582270622253


Epoch 1:   8%|▊         | 2/26 [00:03<00:45,  1.90s/it]

Loss: 14.826980590820312


Epoch 1:  12%|█▏        | 3/26 [00:05<00:43,  1.91s/it]

...

Epoch 10:  96%|█████████▌| 25/26 [00:47<00:01,  1.91s/it]

Loss: 0.0840369313955307


Epoch 10: 100%|██████████| 26/26 [00:48<00:00,  1.87s/it]

Loss: 2.171305339970786e-07


Epoch 10 Validation Accuracy: 0.9901477832512315, F1-macro: 0.4975247524752475


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.9901960784313726, F1-macro: 0.4975247524752475


In [ ]:
current_type = 'disqualifying the positive'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 4. Emotional Reasoning

In [ ]:
# Add labels
data1_1_labels = list(data1['emotional reasoning'][data1_1.index])
data2_1_labels = list(data2['emotional reasoning'][data2_1.index])
data3_1_labels = list(data3['emotional reasoning'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 10

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   4%|▍         | 1/26 [00:01<00:47,  1.89s/it]

Loss: 17.062763214111328


Epoch 1:   8%|▊         | 2/26 [00:03<00:45,  1.90s/it]

Loss: 3.0802483558654785


Epoch 1:  12%|█▏        | 3/26 [00:05<00:43,  1.90s/it]

...

Epoch 10:  96%|█████████▌| 25/26 [00:47<00:01,  1.91s/it]

Loss: 0.8008947372436523


Epoch 10: 100%|██████████| 26/26 [00:48<00:00,  1.87s/it]

Loss: 0.6787471771240234


Epoch 10 Validation Accuracy: 0.8325123152709359, F1-macro: 0.4543010752688172


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8137254901960784, F1-macro: 0.4543010752688172


In [ ]:
current_type = 'emotional reasoning'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 5. Fortune Telling

In [ ]:
# Add labels
data1_1_labels = list(data1['fortune telling'][data1_1.index])
data2_1_labels = list(data2['fortune telling'][data2_1.index])
data3_1_labels = list(data3['fortune telling'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 10

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   4%|▍         | 1/26 [00:01<00:47,  1.88s/it]

Loss: 1.2807241678237915


Epoch 1:   8%|▊         | 2/26 [00:03<00:45,  1.90s/it]

Loss: 4.526332378387451


Epoch 1:  12%|█▏        | 3/26 [00:05<00:43,  1.90s/it]

...

Epoch 10:  96%|█████████▌| 25/26 [00:47<00:01,  1.91s/it]

Loss: 0.798467218875885


Epoch 10: 100%|██████████| 26/26 [00:48<00:00,  1.87s/it]

Loss: 0.37465932965278625


Epoch 10 Validation Accuracy: 0.8620689655172413, F1-macro: 0.4960992907801418


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8823529411764706, F1-macro: 0.4960992907801418


In [ ]:
current_type = 'fortune telling'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 6. Labeling

In [ ]:
# Add labels
data1_1_labels = list(data1['labeling'][data1_1.index])
data2_1_labels = list(data2['labeling'][data2_1.index])
data3_1_labels = list(data3['labeling'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 10

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   4%|▍         | 1/26 [00:01<00:47,  1.88s/it]

Loss: 1.6382334232330322


Epoch 1:   8%|▊         | 2/26 [00:03<00:45,  1.90s/it]

Loss: 5.039063930511475


Epoch 1:  12%|█▏        | 3/26 [00:05<00:43,  1.91s/it]

...

Epoch 10:  96%|█████████▌| 25/26 [00:47<00:01,  1.91s/it]

Loss: 0.3551477789878845


Epoch 10: 100%|██████████| 26/26 [00:48<00:00,  1.87s/it]

Loss: 0.09348882734775543


Epoch 10 Validation Accuracy: 0.8029556650246306, F1-macro: 0.46886446886446886


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8480392156862745, F1-macro: 0.46886446886446886


In [ ]:
current_type = 'labeling'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 7. Magnification

In [ ]:
# Add labels
data1_1_labels = list(data1['labeling'][data1_1.index])
data2_1_labels = list(data2['labeling'][data2_1.index])
data3_1_labels = list(data3['labeling'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 10

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   4%|▍         | 1/26 [00:01<00:47,  1.89s/it]

Loss: 2.5669641494750977


Epoch 1:   8%|▊         | 2/26 [00:03<00:45,  1.90s/it]

Loss: 4.225327968597412


Epoch 1:  12%|█▏        | 3/26 [00:05<00:43,  1.90s/it]

...

Epoch 10:  96%|█████████▌| 25/26 [00:47<00:01,  1.91s/it]

Loss: 1.4486496448516846


Epoch 10: 100%|██████████| 26/26 [00:48<00:00,  1.87s/it]

Loss: 1.1349002122879028


Epoch 10 Validation Accuracy: 0.8325123152709359, F1-macro: 0.5064359267734554


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8676470588235294, F1-macro: 0.5064359267734554


In [ ]:
current_type = 'magnification'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 8. Mind Reading

In [ ]:
# Add labels
data1_1_labels = list(data1['mind reading'][data1_1.index])
data2_1_labels = list(data2['mind reading'][data2_1.index])
data3_1_labels = list(data3['mind reading'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 10

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   4%|▍         | 1/26 [00:01<00:47,  1.88s/it]

Loss: 2.9873878955841064


Epoch 1:   8%|▊         | 2/26 [00:03<00:45,  1.90s/it]

Loss: 16.783336639404297


Epoch 1:  12%|█▏        | 3/26 [00:05<00:43,  1.90s/it]

...

Epoch 10:  96%|█████████▌| 25/26 [00:47<00:01,  1.91s/it]

Loss: 0.7020993232727051


Epoch 10: 100%|██████████| 26/26 [00:48<00:00,  1.87s/it]

Loss: 0.673779308795929


Epoch 10 Validation Accuracy: 0.7586206896551724, F1-macro: 0.5512294157455448


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.803921568627451, F1-macro: 0.5512294157455448


In [ ]:
current_type = 'mind reading'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 9. Overgeneralizing

In [ ]:
# Add labels
data1_1_labels = list(data1['overgeneralizing'][data1_1.index]) # data1에서는 명칭이 다름.
data2_1_labels = list(data2['overgeneralization'][data2_1.index])
data3_1_labels = list(data3['overgeneralization'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 10

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   4%|▍         | 1/26 [00:01<00:42,  1.71s/it]

Loss: 1.657505750656128


Epoch 1:   8%|▊         | 2/26 [00:03<00:40,  1.68s/it]

Loss: 6.536026954650879


Epoch 1:  12%|█▏        | 3/26 [00:05<00:38,  1.67s/it]

...

Epoch 10:  96%|█████████▌| 25/26 [00:47<00:01,  1.92s/it]

Loss: 1.2493101358413696


Epoch 10: 100%|██████████| 26/26 [00:48<00:00,  1.87s/it]

Loss: 0.5622022151947021


Epoch 10 Validation Accuracy: 0.8177339901477833, F1-macro: 0.5548509452972205


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8186274509803921, F1-macro: 0.5548509452972205


In [ ]:
current_type = 'overgeneralizing'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 10. Should Statements

In [ ]:
# Add labels
data1_1_labels = list(data1['should statements'][data1_1.index])
data2_1_labels = list(data2['should statements'][data2_1.index])
data3_1_labels = list(data3['should statements'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 10

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   4%|▍         | 1/26 [00:01<00:47,  1.89s/it]

Loss: 1.4640066623687744


Epoch 1:   8%|▊         | 2/26 [00:03<00:45,  1.91s/it]

Loss: 2.801388740539551


Epoch 1:  12%|█▏        | 3/26 [00:05<00:43,  1.91s/it]

...

Epoch 10:  96%|█████████▌| 25/26 [00:47<00:01,  1.91s/it]

Loss: 0.42217063903808594


Epoch 10: 100%|██████████| 26/26 [00:48<00:00,  1.87s/it]

Loss: 0.5678803324699402


Epoch 10 Validation Accuracy: 0.8866995073891626, F1-macro: 0.5437310661585069


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.9117647058823529, F1-macro: 0.5437310661585069


In [ ]:
current_type = 'should statements'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 11. Mental Filter

In [ ]:
# Add labels
data1_1_labels = list(data1['mental filter'][data1_1.index])
data2_1_labels = list(data2['mental filter'][data2_1.index])
data3_1_labels = list(data3['mental filter'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 10

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   4%|▍         | 1/26 [00:01<00:47,  1.89s/it]

Loss: 0.6224747896194458


Epoch 1:   8%|▊         | 2/26 [00:03<00:45,  1.90s/it]

Loss: 3.9320268630981445


Epoch 1:  12%|█▏        | 3/26 [00:05<00:43,  1.90s/it]

...

Epoch 10:  96%|█████████▌| 25/26 [00:47<00:01,  1.91s/it]

Loss: 0.19576497375965118


Epoch 10: 100%|██████████| 26/26 [00:48<00:00,  1.87s/it]

Loss: 0.46493667364120483


Epoch 10 Validation Accuracy: 0.9014778325123153, F1-macro: 0.4740932642487047


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.9117647058823529, F1-macro: 0.4740932642487047


In [ ]:
current_type = 'mental filter'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 12. Personalization and Blaming

In [ ]:
# Add labels
data1_1_labels = list(data1['personalization and blaming'][data1_1.index])
data3_1_labels = list(data3['personalization and blaming'][data3_1.index]) # data2에는 없음.

# Merging Data
data_encoded = data1_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 10

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:  17%|█▋        | 1/6 [00:01<00:08,  1.77s/it]

Loss: 3.502189874649048


Epoch 1:  33%|███▎      | 2/6 [00:03<00:07,  1.75s/it]

Loss: 18.072551727294922


Epoch 1:  50%|█████     | 3/6 [00:05<00:05,  1.75s/it]

...

Epoch 10:  83%|████████▎ | 5/6 [00:09<00:01,  1.88s/it]

Loss: 1.450488567352295


Epoch 10: 100%|██████████| 6/6 [00:10<00:00,  1.70s/it]

Loss: 1.0246431827545166


Epoch 10 Validation Accuracy: 0.813953488372093, F1-macro: 0.6126126126126126


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8, F1-macro: 0.6126126126126126


In [ ]:
current_type = 'personalization and blaming'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# Accuracy

In [ ]:
results_df

,distortion_type,test_accuracy,f1_macro
0,all-or-nothing thinking,0.848039,0.488970
1,comparing and despairing,0.840000,0.456522
2,disqualifying the positive,0.990196,0.497537
3,emotional reasoning,0.813725,0.534470
4,fortune telling,0.882353,0.468750
5,labeling,0.848039,0.612856
6,magnification,0.867647,0.498863
7,mind reading,0.803922,0.598030
8,overgeneralizing,0.818627,0.498372
9,should statements,0.911765,0.567593
